# Tackling MNIST with just Numpy 

### Installing dependecies
Note: Tensorflow is imported here so that we can easily get the mnist dataset. 

In [ ]:
import pandas as pd
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt

### Loading Data and preprocessing

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
print (f'x_train shape: {x_train.shape}')
print (f'y_train shape: {y_train.shape}')
print (f'y_train example: {y_train[0]}')

In [ ]:
%matplotlib inline

fig, axes = plt.subplots(2,5, figsize=(12,5))
axes = axes.flatten()
idx = np.random.randint(0,60000,size=10)
for i in range(10):
    axes[i].imshow(x_train[idx[i],:].reshape(28,28), cmap='gray')
    axes[i].axis('off') # hide the axes ticks
    axes[i].set_title(str(int(y_train[idx[i]])), color= 'black', fontsize=25)
plt.show()

In [ ]:
'''
Each pixel is single grayscale, meaning a value between 0 to 255.
Normalising the inputs to a value between 0 to 1 makes the calculations later on smaller.
This makes the gradient value closer to our weight initialization 
'''

x_train = x_train.reshape(x_train.shape[0], x_train.shape[1]*x_train.shape[2])
x_train = x_train/255
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1]*x_test.shape[2])
x_test = x_test/255

def one_hot_encode_array(y:np.array):
    '''Hot encodes the output examples.'''
    hot_encoded = np.zeros([y.shape[0],10 ])
    for i, x in enumerate(y):
        hot_encoded[i][x] = 1
    return hot_encoded

y_train = one_hot_encode_array(y_train)
y_test = one_hot_encode_array(y_test)

In [ ]:

def softmax(x):
    """Compute the softmax of vector x in a numerically stable way."""
    shiftx = x - np.max(x,axis=0)
    exps = np.exp(shiftx)
    return exps / np.sum(exps,axis=0)

def d_softmax(x):
    shiftx = x - np.max(x,axis=0)
    exps = np.exp(shiftx)
    return exps/(np.sum(exps,axis=0) * (1-exps/np.sum(exps,axis=0)))

def relu(x):
    x[x<0] = 0
    return x 

def d_relu(x):
    return x >0

def forward_pass(a_0, w_1, w_2):
    z_1 = w_1 @ a_0   #   N1 x 784 * 784 x m  = N1 x m
    a_1 = relu(z_1) #  N1 x m 
    z_2 = w_2 @ a_1 #  10 x N1 * N1 x m  = 10 x m
    a_2 = softmax(z_2) # 10 x m
    return z_1,a_1, z_2 ,a_2

def cost_function(x,y):
    return (x - y) ** 2

def d_cost_function(a_2,y):
    return ( a_2 - y) # The 2 here is irrelavent in the grand scheme of things. This is because the learning rate will be a constant multiple as well.
    
def backward_pass(a_0, z_1,a_1, z_2 ,a_2, w_1,w_2, Y):
    # Backward pass, differenciate wrt Cost function.
    d_a2 =  d_cost_function(a_2, Y) # 10 x m
    d_z2 = d_cost_function(a_2, Y) # 10 x m
    d_w2 = d_z2 @ a_1.T # 10 x m * m x N1 = 10 x N1
 
    d_a1 = w_2.T @ d_z2 # N1 x 10  * 10 x m  = N1 x m
    d_z1 = d_a1 * d_relu(z_1) # N1 x m   N1 x m = N1 x m
    d_w1 = d_z1 @ a_0.T # N1 x m * m x 784 = N1 x 784

    assert d_w1.shape == w_1.shape, f'{d_w1.shape} {w_1.shape}'
    assert d_w2.shape == w_2.shape, f'{d_w2.shape} {w_2.shape}'

    return  d_w1 , d_w2

def update_weights(w_1,w_2, d_w1 , d_w2, learning_rate, N):
    w_1 = w_1 - learning_rate*d_w1 / N
    w_2 = w_2 - learning_rate*d_w2 / N
    return w_1, w_2

In [ ]:
def gradient_descent(X, Y, w_1, w_2 , epochs = 1, learning_rate=0.001):
    for i in range(epochs):
        N = X.shape[0]
        a_0 = X
        z_1, a_1, z_2 ,a_2 = forward_pass(a_0, w_1, w_2)
        print(z_2)
        d_w1 , d_w2 = backward_pass(a_0, z_1,a_1, z_2 ,a_2, w_1,w_2, Y)
        w_1, w_2 = update_weights(w_1,w_2, d_w1,d_w2,learning_rate,N)

    return w_1,w_2

def get_prediction(a_2):
    return np.argmax(a_2.T)

def check_accuracy(X,Y ,w_1,w_2):
    a_0 = X
    z_1,a_1, z_2 ,a_2 = forward_pass(a_0, w_1, w_2)
    return a_2

In [ ]:
neurons = 256
epochs = 100
learning_rate = 0.001
w_1 = np.random.rand(neurons,784)
w_2 = np.random.rand(10, neurons)
orig_w_1 = w_1.copy()

In [141]:
w_1,w_2 = gradient_descent(x_train.T, y_train.T, w_1, w_2, epochs, learning_rate)

[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]]


In [ ]:
output = check_accuracy(x_test.T,y_test.T,w_1,w_2)
np.argmax(output, axis=0)

In [ ]:
np.argmax(y_test,axis=1)